In [1]:
import os
import brunoflow as bf
import jax
from jax import numpy as jnp
import numpy as np
import unittest as ut
jax.config.update("jax_enable_x64", True)
print(f"Running JAX on {jax.devices()[0].device_kind}")

Running JAX on NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [2]:
from brunoflow.test import utils
inputs = utils.random_inputs([[(6, 5)]])
fn=lambda x: x[1:3, 1]
torch_fn=lambda x: x[1:3, 1]

In [3]:
inputs

In [4]:
for inp in inputs:
    x = utils.inputs_to_params(inp)
    x_t = utils.inputs_to_torch(inp)
    y = fn(*x)
    y_t = torch_fn(*x_t)
    utils.check(ut.TestCase(), y, y_t)

In [9]:
import numpy as np
a = bf.Parameter(np.ones((10, 10)))
np.zeros_like(a.val)

# a

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [5]:
x, x_t

([node(name: None, val: [[-0.35777008 -0.11910278  1.18416186  0.04687563 -1.92564693]
   [ 1.09005847  0.34922955  0.62983513  0.2621987   1.32636705]
   [-0.57965284  2.18238234 -0.28184199  0.29220806 -0.94973982]
   [-0.40123923  0.09223514 -0.27802103 -1.4646394   1.1170621 ]
   [ 0.15621939 -0.18970046 -1.30898383 -0.69910879 -0.23543655]
   [-1.29996201  1.96826118 -0.17853028 -1.53830553  0.93935072]], grad: [[0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]])],
 [tensor([[-0.3578, -0.1191,  1.1842,  0.0469, -1.9256],
          [ 1.0901,  0.3492,  0.6298,  0.2622,  1.3264],
          [-0.5797,  2.1824, -0.2818,  0.2922, -0.9497],
          [-0.4012,  0.0922, -0.2780, -1.4646,  1.1171],
          [ 0.1562, -0.1897, -1.3090, -0.6991, -0.2354],
          [-1.3000,  1.9683, -0.1785, -1.5383,  0.9394]], dtype=torch.float64)])